周内效应——左侧买入

In [ ]:
import pandas as pd
from datetime import datetime
import trdb2py
import numpy as np

isStaticImg = False
width = 960
height = 768

pd.options.display.max_columns = None
pd.options.display.max_rows = None

trdb2cfg = trdb2py.loadConfig('./trdb2.yaml')

In [ ]:
# 具体基金
asset = 'jqdata.000300_XSHG|1d'
# baselineasset = 'jrj.510310'
# baselineasset = 'jqdata.000300_XSHG|1d'

# 起始时间，0表示从最开始算起
tsStart = 0
tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

# 初始资金池
paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

# 买入参数，用全部的钱来买入（也就是复利）
paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

# 买入参数，用全部的钱来买入（也就是复利）
paramsbuy2 = trdb2py.trading2_pb2.BuyParams(
    moneyParts=2,
)

# 卖出参数，全部卖出
paramssell = trdb2py.trading2_pb2.SellParams(
    perVolume=1,
)

# 卖出参数，全部卖出
paramssell7 = trdb2py.trading2_pb2.SellParams(
#     perVolume=1,
    keepTime=7 * 24 * 60 * 60,
)

lststart = [1, 2, 3, 4, 5]
lsttitle = ['周一', '周二', '周三', '周四', '周五']

In [ ]:
def calcweekday2val2(wday, offday):
    if offday == 1:
        if wday == 5:
            return 3
    if offday == 2:
        if wday >= 4:
            return 4
    if offday == 3:
        if wday >= 3:
            return 5
    if offday == 4:
        if wday >= 2:
            return 6
        
    return offday


前几天有朋友提到，咱们的策略，是在均线上下分开判断，拿当天收盘价来计算均线，但这个价格出现后，理论上是无法按这个价格买到的。

这其实就是左侧交易和右侧交易的问题。

顾名思义，左侧交易是在信号还未达成时但预期信号即将达成时交易，而右侧交易指在信号达成以后交易。
拿咱们的策略来看，左侧交易应该是收盘价还未出来时，我们预判今天收盘价，然后计算一个indicator，用这个指导交易。
而右侧交易，则是等收盘价出来，在第二天执行交易。

我们先来尝试将右侧交易转换为左侧交易。

我们的目标策略如下：

1. 如果今天是周四，且当日收盘价在 ema 29 日线上方，则买入，并于下周周三无条件卖出。
2. 如果今天是周一，且当日收盘价在 ema 29 日线下方，则买入，并于下周周五无条件卖出。
3. 如果周一已经买入，则周四不操作，反之亦然。

国内的基金交易策略有2种，分别是场内ETF和场外基金，场内交易价格实时变化，而且还有交易量的问题，比较复杂。
而场外交易相对简单，就是交易日下午3点以前买入，按当日价格结算份额，下午3点以后买入，则按下一个交易日价格结算（周五下午3点后买入，按周一价格结算，QDII基金一般还要再延后一天）。

所以我们先按场外交易规则来执行（后面来讲更复杂的场内交易规则）。

因为我们是买指数，所以能得到实时的指数价格，如果我们拿接近下午3点的指数价格，来计算均线，虽然有误差，但感觉也许还是能接受的。
这里还需要考虑得到信号以后的购买时间问题，所以暂时拿5分钟数据测试看看。

因为咱们是算法回测，所以干脆多测一些情况，把下午2点到3点之间，每间隔5分钟的情况都测一下。

In [ ]:
# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='沪深300指数',
)  

pnlBaseline1 = trdb2py.simTrading(trdb2cfg, p0)

trdb2py.showPNLs([pnlBaseline1], toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []

lstassetcode = ['jqdata.000300_XSHG|1d']
lstassetname = ['沪深300混合策略']

# tsStart = 0
# tsEnd = -1

for ai in range(0, len(lstassetcode)):
    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(lstassetcode[ai]),         
    )

    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[4, calcweekday2val2(4, 4)],
    )

    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['up'],
        strVals=['ta-ema.{}'.format(29)],
    )

    buy2 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[1, calcweekday2val2(1, 4)],
        group=1,
    )

    buy3 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['down'],
        strVals=['ta-ema.{}'.format(29)],
        group=1,    
    )

    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[3],
    )

    sell1 = trdb2py.trading2_pb2.CtrlCondition(
        name='ctrlconditionid',
        vals=[1],
        strVals=['buy'],
    )

    sell2 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[5],
        group=1,     
    )

    sell3 = trdb2py.trading2_pb2.CtrlCondition(
        name='ctrlconditionid',
        vals=[2],
        strVals=['buy'],
        group=1,     
    )

    paramsbuy = trdb2py.trading2_pb2.BuyParams(
        perHandMoney=1,
    )

    paramsinit = trdb2py.trading2_pb2.InitParams(
        money=10000,
    )
    
    feebuy = trdb2py.trading2_pb2.FeeParams(
        percentage=0.0003,
    )
    
    feesell = trdb2py.trading2_pb2.FeeParams(
        percentage=0.0003,
    )    

    s0.buy.extend([buy0, buy1, buy2, buy3])
    s0.sell.extend([sell0, sell1, sell2, sell3])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
#     s0.feeBuy.CopyFrom(feebuy)        
#     s0.feeSell.CopyFrom(feesell)            
    p0 = trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(lstassetcode[ai]), trdb2py.str2asset('jqdata.000300_XSHG|5m')],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title=lstassetname[ai],
    )  

    lstparams.append(p0)

lstpnl1 = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs(lstpnl1 + [pnlBaseline1], toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []

lstassetcode = ['jqdata.000300_XSHG|1d',
                'jqdata.000300_XSHG|1d',
                'jqdata.000300_XSHG|1d',
                'jqdata.000300_XSHG|1d',
                'jqdata.000300_XSHG|1d',
                'jqdata.000300_XSHG|1d',
                'jqdata.000300_XSHG|1d',
                'jqdata.000300_XSHG|1d',
                'jqdata.000300_XSHG|1d',
                'jqdata.000300_XSHG|1d',
                'jqdata.000300_XSHG|1d',
                'jqdata.000300_XSHG|1d']
lstassetname = ['沪深300左侧交易14:00',
                '沪深300左侧交易14:05',
                '沪深300左侧交易14:10',
                '沪深300左侧交易14:15',
                '沪深300左侧交易14:20',
                '沪深300左侧交易14:25',
                '沪深300左侧交易14:30',
                '沪深300左侧交易14:35',
                '沪深300左侧交易14:40',
                '沪深300左侧交易14:45',
                '沪深300左侧交易14:50',
                '沪深300左侧交易14:55']
lsttimeparam = [50400,
                50700,
                51000,
                51300,
                51600,
                51900,
                52200,
                52500,
                52800,
                53100,
                53400,
                53700]

# tsStart = 0
# tsEnd = -1

for ai in range(0, len(lstassetcode)):
    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(lstassetcode[ai]),         
    )

    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[4, calcweekday2val2(4, 4)],
    )

    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['up'],
        strVals=['ta-ema.{}>day/1d/5m/{}'.format(29, lsttimeparam[ai])],
    )

    buy2 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[1, calcweekday2val2(1, 4)],
        group=1,
    )

    buy3 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['down'],
        strVals=['ta-ema.{}>day/1d/5m/{}'.format(29, lsttimeparam[ai])],
        group=1,    
    )

    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[3],
    )

    sell1 = trdb2py.trading2_pb2.CtrlCondition(
        name='ctrlconditionid',
        vals=[1],
        strVals=['buy'],
    )

    sell2 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[5],
        group=1,     
    )

    sell3 = trdb2py.trading2_pb2.CtrlCondition(
        name='ctrlconditionid',
        vals=[2],
        strVals=['buy'],
        group=1,     
    )

    paramsbuy = trdb2py.trading2_pb2.BuyParams(
        perHandMoney=1,
    )

    paramsinit = trdb2py.trading2_pb2.InitParams(
        money=10000,
    )
    
    feebuy = trdb2py.trading2_pb2.FeeParams(
        percentage=0.0003,
    )
    
    feesell = trdb2py.trading2_pb2.FeeParams(
        percentage=0.0003,
    )    

    s0.buy.extend([buy0, buy1, buy2, buy3])
    s0.sell.extend([sell0, sell1, sell2, sell3])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
#     s0.feeBuy.CopyFrom(feebuy)        
#     s0.feeSell.CopyFrom(feesell)            
    p0 = trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(lstassetcode[ai]), trdb2py.str2asset('jqdata.000300_XSHG|5m')],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title=lstassetname[ai],
    )  

    lstparams.append(p0)

lstpnl3 = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs(lstpnl3 + lstpnl1 + [pnlBaseline1], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport(lstpnl3 + lstpnl1)

dfpnl[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

从回报曲线图上可以看到，一开始，各种策略都相差不大，时间越长才越明显。

表格会更直观一些，在下午2点10分以后，策略几乎趋近一致。

原因其实也很容易能想清楚，ema 29 日均线，最后一天的数据偏差一点点，本来就影响不大。

所以，我们只需要在周一或周四的下午2点半以后，拿当天的数据简单计算一下均线，即可做出是否单日买入的决策。

这里还有需要注意的是，因为这个策略和29日均线有关，所以前29天是不能操作的，大家看前面的图也能看出来，这点大家一定要注意，如果一定要操作，最好拿正式数据填充进去。